In [1]:
import os

In [2]:
foo = 'D:/results_bender/german_classify/classifications_train.csv'

In [4]:
directory = os.path.dirname(foo)

In [9]:
directory 

'D:/results_bender/german_classify'

In [5]:
if foo:
    print("yes")

yes


In [6]:
n = None

In [8]:
if not n:
    print("yes")
else:
    print("None")

yes


In [10]:
os.path.exists(None)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [20]:
import numpy as np
import pandas as pd

In [21]:
d = {
    'a': [2,3,1,5],
    'b': [5,7,8,3],
    'c': [8,4,6,5],
    'd': [8,7,8,6],
    'e': [1,1,1,1],
    'f': [2,2,2,2],
    'g': [4,5,6,7]
}

In [22]:
df = pd.DataFrame(d)

In [23]:
df.head()

,a,b,c,d,e,f,g
0,2,5,8,8,1,2,4
1,3,7,4,7,1,2,5
2,1,8,6,8,1,2,6
3,5,3,5,6,1,2,7


In [28]:
first = list("ade")
second = list("bcg")

In [29]:
first

['a', 'd', 'e']

In [30]:
first + second

['a', 'd', 'e', 'b', 'c', 'g']

In [37]:
df[df.columns.intersection(first+second)]

,a,b,c,d,e,g
0,2,5,8,8,1,4
1,3,7,4,7,1,5
2,1,8,6,8,1,6
3,5,3,5,6,1,7


In [2]:
foo = np.array([23, 12, 5, 19])

In [6]:
np.savetxt('test_numpy.txt', foo, fmt='%0.10g')

In [4]:
print(foo)

[23 12  5 19]


In [15]:
import pandas as pd

In [16]:
foo

array([23, 12,  5, 19])

In [19]:
foo_2 = pd.DataFrame(foo)

In [18]:
foo.shape

(4,)

In [ ]:
foo_2.to_csv()

# To do:

- build dataset
- test `AcousticDatafile(test.wav)` and `segments` method
- test `AcousticDataset(directory)` and `segments` method

In [7]:
from dnnseg.data import AcousticDatafile, AcousticDataset

In [8]:
import os

In [9]:
os.chdir("D:/model/german/")

In [10]:
file_names = [x for x in os.listdir() if x.endswith(".wav")][:50]

In [11]:
dataset = AcousticDataset("D:/tests/")

Processing file "de2_e_b..." 7/7       |    ETA - 0s     


In [14]:
dataset.labels(one_hot=False, segment_type='wrd').ndim

1

Solving issue in .inputs and .targets methods inside AcousticDataset class.
Notes: `.inputs` and `.targets` only call `.segment_and_stack`, once as `padding = 'pre'` and once `padding = 'post'`
# To do:
- test `.segment_and_stack`from `AcousticDataset` outside of script 
- test `.inputs` and `.targets` outside of script
- UNDERSTAND what this does

In [ ]:
dataset.targets()

In [ ]:
dataset.segment_and_stack()

In [ ]:
dataset.inputs()

In [ ]:
datafile = dataset.data["de2_e_b01_v0102301011114"]

In [ ]:
datafile.segment_and_stack(padding="pre")

In [ ]:
datafile.segment_and_stack(padding="post")

`segment_and_stack` method from AcousticDatafile

Arguments passed during fitting:
```
segments = "wrd",
max_len = 50,
padding = "pre",
reverse = False,
normalize = False,
center = True,
with_deltas = True,
resample = None
```

In [ ]:
segments_arr = datafile.wrd_segments

In [ ]:
segments_arr

In [ ]:
padding

In [ ]:
pad_seqs = padding not in ['None', None]

In [ ]:
pad_seqs

In [ ]:
from dnnseg.data import pad_sequence

In [ ]:
import numpy as np

In [ ]:
bounds = np.array(np.rint((segments_arr[['start', 'end']] / datafile.offset)),
                  dtype=np.int32)

In [ ]:
bounds

`bounds` were the boundaries in which there is voice in ms * 1000 / 10 (because the offset is 10), now corrected

In [ ]:
bounds.shape

In [ ]:
datafile.offset

In [ ]:
import scipy

In [ ]:
segments = "wrd"
max_len = 1300
padding = "pre"
reverse = False
normalize = False
center = True
with_deltas = True
resample = None

feats_split = []
mask_split = []

print(f"feats = {datafile.feats}")
print(f"feats shape is {datafile.feats.shape}")

for i in range(len(bounds)):
    print(f"loop no. {i}")
    s = bounds[i, 0]
    e = bounds[i, 1]
    print(f"s = {s}")
    print(f"e = {e}")
    
    if max_len is not None:
        print("cutting to maximum length")
        # cut features to maximum length
        e = min(e, s + max_len)
    new_feats = datafile.feats[s:e, :]
    length = e - s
    print(f"new_feats = {new_feats}")
    print(f"new_feats shape is {new_feats.shape}")
    print(f"length = {length}")
    
    
    if resample and new_feats.shape[0] > 0:
        print("resampling")
        length = resample
        new_feats = scipy.signal.resample(new_feats, resample, axis=0)
        print(f"new_feats = {new_feats}")

    if not with_deltas:
        print("cutting deltas")
        new_feats = new_feats[:, :datafile.n_coef]
        print(f"new_feats = {new_feats}")

    if normalize:
        print("normalizing")
        maximum = new_feats.max()
        minimum = new_feats.min()
        diff = maximum - minimum
        new_feats = (new_feats - minimum) / diff
        print(f"new_feats = {new_feats}")
    if center:
        print("before centering")
        print(f"new_feats = {new_feats}")
        print(f"new_feats.mean = {new_feats.mean()}")
        print(f"Are feats and mean the same? --> {new_feats == new_feats.mean()}")
        new_feats = new_feats - new_feats.mean()
        print("after centering")
        print(f"new_feats = {new_feats}")

    if not pad_seqs:
        print("pad_seqs is None")
        new_feats = new_feats[None, ...]
        mask = np.ones((1, length))
        print(f"new_feats = {new_feats}")
    else:
        print(f"creating mask with length of {length}")
        mask = np.ones(length)
        print(f"new_feats = {new_feats}")

    feats_split.append(new_feats)
    mask_split.append(mask)
    
    print("feats_split:")
    print(feats_split)
    print("\n")
    print("feats length:")
    print(len(feats_split))
    print("\n")
    print("mask_split:")
    print(mask_split)
    print("\n")
    print("mask length:")
    print(len(mask_split))
    

if pad_seqs:
    feats = pad_sequence(feats_split, padding=padding, reverse=reverse)
    mask = pad_sequence(mask_split, padding=padding, reverse=reverse)
else:
    feats = feats_split
    mask = mask_split

In [ ]:
datafile.feats[500:502]

In [ ]:
datafile.feats.shape

In [ ]:
datafile.n_coef

In [ ]:
type(s)

In [ ]:
new_feats = datafile.feats - datafile.feats.mean()

In [ ]:
new_feats.shape

## error causes found:

1) `max_len` needs to be adapted to this material in config file. Originally it was 50, which yields an empty array at `segment_and_stack`. It was supposed to be the maximum syllable length in the material set.

2) `self.bounds`, the boundaries in which there is voice in ms * 1000 / offset (offset is defined by `self.offset`), are yielding a slice of `self.features` that is empty. `self.features.shape` is `(149, 39)`, the boundaries are being set at over 5000 

## to do:
- find maximum syllable length in material set (`D:/MASTER/23-LIN-MaCL1/Modulprüfung/dfc3sd.csv`)
- set found length (in ms) in config file `max_len`
- correct boundaries method to yield boundaries under 149 

In [ ]:
feats.shape

In [ ]:
mask.shape

In [ ]:
dataset.labels()

In [ ]:
from dnnseg.backend import binary2integer

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
test = tf.Session()

In [ ]:
b2i = binary2integer(b=np.array([[True, True, False],[True, True, False]]), session=test)

In [ ]:
b2i

In [ ]:
with test:
    b2i.eval()

In [ ]:
b2i.value_index

In [ ]:
np.array([[True, True, False],[True, True, False]]).shape

In [ ]:
x, x_mask = dataset.inputs()

In [ ]:
indices = np.arange(0, min(0 + 256, len(dataset.targets()[1])))

In [ ]:
indices

In [ ]:
x.shape

In [ ]:
x[indices]

In [ ]:
x_mask[indices]

In [ ]:
x, x_mask = dataset.segment_and_stack()

In [ ]:
x.shape

In [ ]:
dataset.data

In [ ]:
x, x_mask = datafile.segment_and_stack(padding="pre", max_len=30)

In [ ]:
x.shape

In [ ]:
x, x_mask = dataset.segment_and_stack(max_len=30)

In [ ]:
x.shape

# to do:
- read sampa_to_ipa.csv correctly
- change read method in plot.py

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("D:/model/german_sampa_to_ipa.csv", sep=";")

In [ ]:
df.head()

In [ ]:
imp